In [1]:
import pandas as pd
import pickle
import pickle
from pathlib import Path

convin_data_path = Path("convin_data/convin_complete_dataset_no_embedding.pkl")

df = pd.read_pickle(convin_data_path)

In [4]:
df.columns

Index(['roman_transcript', 'native_transcript', 'machine_transcript',
       'emotion', 'fine-grained-emotion'],
      dtype='object')

In [6]:
df["fine-grained-emotion"].value_counts()

fine-grained-emotion
None of the above    1691
Junk                 1319
Curiosity            1126
Confusion              89
Realization            89
Anticipation            2
Surprise                1
Name: count, dtype: int64

In [3]:
df.sample(1)

,domain,utterance_file,domain_batch,context_file,emotion,fine-grained-emotion,text_embedding,audio_embedding,opensmile_embedding,machine_transcript,roman_transcript,native_transcript
3110,medfin,label_studio_data/CONVIN/medfin_batch2/utteran...,medfin_batch2,label_studio_data/CONVIN/medfin_batch2/context...,Neutral,None of the above,"[[[-0.5146, 0.3608, -2.084, 1.298, 0.07574, -0...","[[[0.94190216, -0.4180501, 0.9677263, 0.426029...","[['unknown', 2.044529, 0.6833603, 0.631664, 0....",कॊ हमनॅ दॅखा है ऒकॅ यॅ शहि है,ko hamane dekha hai oke ye sahi hai,को हमने देखा है ಓಕೆ ये सही है


In [109]:
sample_utt = df# .sample(10)

import ast
import numpy as np

# import torch pad_sequence
from torch.nn.utils.rnn import pad_sequence
import torch

torch_data = []
for x in sample_utt["text_embedding"].tolist():
    if x is not None:
        try:
            torch_data.append(torch.tensor(x).squeeze())
        except:
            print(x)
            break

# torch_data = [torch.tensor(x).squeeze() for x in sample_utt["text_embedding"].tolist()]
print(torch_data[0].shape)
pad_sequence(torch_data, batch_first=True).shape

torch.Size([18, 4096])


torch.Size([4269, 4096, 4096])

In [159]:
!export PYDEVD_WARN_SLOW_RESOLVE_TIMEOUT=20

5367.02s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


In [160]:
import os

os.environ["PYDEVD_WARN_SLOW_RESOLVE_TIMEOUT"] = "20"

In [162]:
# sample_utt = df

batches = set()
dialogue_ids = set()
utt_ids = set()

df['dialogue_id'] = None
df['utt_id'] = None

for idx, x in enumerate(df['utterance_file'].tolist()):
    utt_path = Path(x)
    try:
        df.loc[idx, 'dialogue_id'] = utt_path.name.split("_")[1]
        df.loc[idx, 'utt_id'] = Path(utt_path.name.split("_")[-1]).stem    
    except Exception as e:
        print(utt_path)

        

df.sample(1)

# utt_path, batch, batch_id, utt_id

,domain,utterance_file,domain_batch,context_file,emotion,fine-grained-emotion,text_embedding,audio_embedding,opensmile_embedding,machine_transcript,roman_transcript,native_transcript,dialogue_id,utt_id
3952,getsimpl,getsimpl_batch1/utterances/getsimpl_1570101_ut...,getsimpl_batch1,getsimpl_batch1/contexts/getsimpl_1570101_cont...,Negative,Junk,"[[[-0.5146, 0.3616, -2.084, 1.297, 0.07574, -0...","[[[0.745173, 1.2565163, 3.5633893, 0.18776838,...",NaN,पहलॅ आप लॊग दॅखॊ कहा शॅ ऑर्डर आया है ऑर कॉल कह...,pahale ap log dekho kaha se order aya hai or c...,पहले आप लोग देखो कहा से order आया है or call क...,1570101,351


In [188]:
df["domain_batch"].value_counts()

domain_batch
batch4             591
batch1             552
medfin_batch1      392
medfin_batch2      384
livpure_batch1     384
batch5             379
getsimpl_batch1    360
getsimpl_batch2    355
livpure_batch2     345
batch2             288
batch3             287
Name: count, dtype: int64

In [222]:
columns = ["domain", "domain_batch", "dialogue_id", "utt_id", 
           "roman_transcript", "native_transcript", "machine_transcript", "emotion", "fine-grained-emotion", 
        #    "text_embedding", "audio_embedding"
           ]
df1 = df[columns]
df1 = df1.set_index(["domain", "domain_batch", "dialogue_id", "utt_id"])
df1.sample(3)

,,,,roman_transcript,native_transcript,machine_transcript,emotion,fine-grained-emotion
domain,domain_batch,dialogue_id,utt_id,,,,,
thomascook,batch4,1209,145,thailand not a nice place bali what else what ...,thailand not a nice place bali what else what ...,थाइलैंड नॉट ऍ नाइश प्लॅश बालि ह्वाट ऍल्श ह्वाट...,Neutral,Curiosity
getsimpl,getsimpl_batch2,2538164,105,simple pay later as your payment credit applic...,simple pay later as your payment credit applic...,शिम्पल पॅ लॅटर ऍश यॊर पॅमॅंट क्रॅडिट ऍप्लिकॅशन...,Neutral,Curiosity
thomascook,batch3,3140,285,kar dijie kya requirements hai vaha pe han ji ...,कर दीजिए क्या requirements है वहा पे हाँ जी आप...,कर दिजिऍ क्या रिक्वायरमॅंट्श है वहा पॅ हाँ जि ...,Neutral,Curiosity


In [220]:
import json

df1.sample(5).to_json("convin_data/convin_complete_dataset.json", orient="table")

In [223]:
df1.to_pickle("convin_data/convin_complete_dataset_no_embedding.pkl")

In [210]:
open("convin_data/sample.txt", "w").write(str(len(df1)))

4

In [187]:
df[df["domain_batch"] == "getsimpl_batch2"][["dialogue_id", "utt_id"]]

,dialogue_id,utt_id
3962,2536630,1
3963,2536630,2
3964,2536630,3
3965,2536630,4
3966,2536630,5
...,...,...
4312,2531556,351
4313,2531556,352
4314,2531556,353
4315,2531556,354


In [ ]:
sample_utt = df# .sample(10)

import ast
import numpy as np

# import torch pad_sequence
from torch.nn.utils.rnn import pad_sequence
import torch

torch_data = []
for x in sample_utt["text_embedding"].tolist():
    if x is not None:
        try:
            torch_data.append(torch.tensor(x).squeeze())
        except:
            print(x)
            break

# torch_data = [torch.tensor(x).squeeze() for x in sample_utt["text_embedding"].tolist()]
print(torch_data[0].shape)
pad_sequence(torch_data, batch_first=True).shape